## This notebook will use taskflowai library, build on top of langchain, langgraph (it is used for more modularity of your project code)

## Installing taskflowai

In [1]:
!pip install taskflowai
!pip install streamlit

### Loading the credentials

In [9]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()


# Retrieve the API keys from environment variables
#openai_api_key = os.getenv("OPENAI_API_KEY")
weather_api_key = os.getenv("WEATHER_API_KEY")
serper_api_key = os.getenv("SERPER_API_KEY")
amadeus_api_key = os.getenv("AMADEUS_API_KEY")
amadeus_api_secret = os.getenv("AMADEUS_API_SECRET")
groq_api_key = os.getenv("GROQ_API_KEY")



Error in callback <function Halo.__init__.<locals>.clean_up at 0x0000022061F137F0> (for post_run_cell), with arguments args (<ExecutionResult object at 220620077f0, execution_count=9 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 22061fdff10, raw_cell="import os
from dotenv import load_dotenv

# Load e.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/mailm/Documents/Production-Ready-Multi-Agent-RAG-Project/notebooks/trip_planner_multi_ai_agent_notebook.ipynb#X20sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: Halo.__init__.<locals>.clean_up() takes 0 positional arguments but 1 was given

Error in callback <function Halo.__init__.<locals>.clean_up at 0x0000022061F65000> (for post_run_cell), with arguments args (<ExecutionResult object at 220620077f0, execution_count=9 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 22061fdff10, raw_cell="import os
from dotenv import load_dotenv

# Load e.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/mailm/Documents/Production-Ready-Multi-Agent-RAG-Project/notebooks/trip_planner_multi_ai_agent_notebook.ipynb#X20sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: Halo.__init__.<locals>.clean_up() takes 0 positional arguments but 1 was given

Error in callback <function Halo.__init__.<locals>.clean_up at 0x0000022061F13E20> (for post_run_cell), with arguments args (<ExecutionResult object at 220620077f0, execution_count=9 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 22061fdff10, raw_cell="import os
from dotenv import load_dotenv

# Load e.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/mailm/Documents/Production-Ready-Multi-Agent-RAG-Project/notebooks/trip_planner_multi_ai_agent_notebook.ipynb#X20sZmlsZQ%3D%3D> result=None>,),kwargs {}:


...

TypeError: Halo.__init__.<locals>.clean_up() takes 0 positional arguments but 1 was given

Error in callback <function Halo.__init__.<locals>.clean_up at 0x0000022061F640D0> (for post_run_cell), with arguments args (<ExecutionResult object at 220620077f0, execution_count=9 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 22061fdff10, raw_cell="import os
from dotenv import load_dotenv

# Load e.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/mailm/Documents/Production-Ready-Multi-Agent-RAG-Project/notebooks/trip_planner_multi_ai_agent_notebook.ipynb#X20sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: Halo.__init__.<locals>.clean_up() takes 0 positional arguments but 1 was given

Error in callback <function Halo.__init__.<locals>.clean_up at 0x0000022061F64160> (for post_run_cell), with arguments args (<ExecutionResult object at 220620077f0, execution_count=9 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 22061fdff10, raw_cell="import os
from dotenv import load_dotenv

# Load e.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/mailm/Documents/Production-Ready-Multi-Agent-RAG-Project/notebooks/trip_planner_multi_ai_agent_notebook.ipynb#X20sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: Halo.__init__.<locals>.clean_up() takes 0 positional arguments but 1 was given

### Importing necessary packages

In [3]:

from taskflowai import Agent, Task, OpenaiModels,GroqModels, WebTools, AmadeusTools, WikipediaTools, set_verbosity
import streamlit as st

set_verbosity(True)

### Defining Agents

In [5]:
# Define Agents
web_researcher = Agent(
    role="Web Researcher",
    goal="Search the web thoroughly for information and write comprehensive reports",
    llm=GroqModels.llama_3_1_70b,
    tools=[
        WebTools.serper_search, 
        WikipediaTools.search_articles, 
        WikipediaTools.search_images
    ]
)

travel_agent = Agent(
    role="Travel Agent",
    goal="Assist users with their travel-related queries",
    llm=GroqModels.llama_3_1_70b,
    tools=[
        WebTools.serper_search, 
        WebTools.get_weather_data, 
        AmadeusTools.search_flights
    ]
)

reporter_agent = Agent(
    role="Travel Report Agent",
    goal="Write comprehensive travel reports",
    llm=GroqModels.llama_3_1_70b,
)


### Defining tasks

In [7]:
# Define Tasks
def research_destination_task(destination, interests):
    return Task.create(
        agent=web_researcher,
        context=f"User's Intended Destination: {destination}\n\nUser's Interests: {interests}\n",
        instruction="""
        Research the given destination using Wikipedia and other tools, collecting both articles and images to prepare
        a rich multi-media markdown response about the destination. Include relevant information related to the user's interests.
        """
    )

def research_events_task(destination, interests, dates):
    return Task.create(
        agent=web_researcher,
        context=f"User's Intended Destination: {destination}\n\nUser's Interests: {interests}\n\nUser's Travel Dates: {dates}\n",
        instruction="""
        Use web tools to research events at the given destination within the specified dates. Include events matching
        the user's interests in addition to general events.
        """
    )

def research_weather_task(destination, dates):
    return Task.create(
        agent=travel_agent,
        context=f"User's Intended Destination: {destination}\n\nUser's Travel Dates: {dates}\n",
        instruction="""
        Research and report weather for the given location and date span. If the dates are more than 10 days ahead, use web tools to
        report average weather for that time of year. Provide a detailed weather summary for the destination.
        """
    )

def research_flights_task(location, destination, dates):
    return Task.create(
        agent=travel_agent,
        context=f"User's Current Location: {location}\n\nUser's Intended Destination: {destination}\n\nUser's Travel Dates: {dates}\n",
        instruction="""
        Use flight research tools to find flight options between the given locations and travel dates. Provide a detailed
        report highlighting the best three options.
        """
    )

def write_final_report_task(location, destination, interests, dates, destination_report, events_report, weather_report, flights_report):
    return Task.create(
        agent=reporter_agent,
        context=f"User is traveling from {location} to {destination} around {dates}. Interests: {interests}. \n\n"
               f"Destination Information: {destination_report}\nEvents Report: {events_report}\n"
               f"Weather: {weather_report}\nFlights: {flights_report}\n",
        instruction="""
        Write a comprehensive travel report in markdown format. Include information about the destination, events, weather,
        and flight options in a cohesive and detailed manner, along with relevant images.
        """
    )

### Give it a try

In [8]:
def main():
    # Get user inputs
    location = input("Where are you flying from?\n").strip()
    destination = input("Where are you going?\n").strip()
    interests = input("What are your interests for this trip?\n").strip()
    dates = input("When are you going to travel?\n").strip()

    # Validate inputs
    if not (location and destination and interests and dates):
        print("All inputs are required. Please try again.")
        return

    # Task executions
    destination_report = research_destination_task(destination, interests)
    print("\n--- Destinatioin Report ---\n")

    events_report = research_events_task(destination, interests, dates)
    print("\n--- Events Report ---\n")

    weather_report = research_weather_task(destination, dates)
    print("\n--- Weather Report ---\n")

    flights_report = research_flights_task(location, destination, dates)
    print("\n--- Fights Report ---\n")


    # Compile the final report
    final_report = write_final_report_task(
        location, destination, interests, dates,
        destination_report, events_report, weather_report, flights_report
    )

    # Print the final report
    print("\n--- Final Trip Report ---\n")
    print(final_report)

if __name__ == "__main__":
    main()


Sending request to llama-3.1-70b-versatile from Groq
You are a Web Researcher. Your goal is to Search the web thoroughly for information and write comprehensive reports.

User's Intended Destination: San Francisco

User's Interests: Technology

-----

You can call the following functions to gather information or perform actions:
- search_articles: 
        Search for Wikipedia articles based on a given query.

        Args:
            query (str): The search query string.
            num_results (int, optional): The maximum number of search results to return. Defaults to 10.

        Returns:
            List[Dict[str, str]]: A list of dictionaries containing detailed information about each search result.
            Each dictionary includes:
                - 'title': The title of the article.
                - 'fullurl': The full URL of the article on Wikipedia.
                - 'snippet': A brief extract or snippet from the article.

        Raises:
            requests.exceptions

TypeError: Halo.__init__.<locals>.clean_up() takes 0 positional arguments but 1 was given

Error in callback <function Halo.__init__.<locals>.clean_up at 0x0000022061F65000> (for post_run_cell), with arguments args (<ExecutionResult object at 22050d0fee0, execution_count=8 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 22050d0ed10, raw_cell="def main():
    # Get user inputs
    location = i.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/mailm/Documents/Production-Ready-Multi-Agent-RAG-Project/notebooks/trip_planner_multi_ai_agent_notebook.ipynb#X15sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: Halo.__init__.<locals>.clean_up() takes 0 positional arguments but 1 was given

Error in callback <function Halo.__init__.<locals>.clean_up at 0x0000022061F13E20> (for post_run_cell), with arguments args (<ExecutionResult object at 22050d0fee0, execution_count=8 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 22050d0ed10, raw_cell="def main():
    # Get user inputs
    location = i.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/mailm/Documents/Production-Ready-Multi-Agent-RAG-Project/notebooks/trip_planner_multi_ai_agent_notebook.ipynb#X15sZmlsZQ%3D%3D> result=None>,),kwargs {}:


...

TypeError: Halo.__init__.<locals>.clean_up() takes 0 positional arguments but 1 was given

Error in callback <function Halo.__init__.<locals>.clean_up at 0x0000022061F640D0> (for post_run_cell), with arguments args (<ExecutionResult object at 22050d0fee0, execution_count=8 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 22050d0ed10, raw_cell="def main():
    # Get user inputs
    location = i.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/mailm/Documents/Production-Ready-Multi-Agent-RAG-Project/notebooks/trip_planner_multi_ai_agent_notebook.ipynb#X15sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: Halo.__init__.<locals>.clean_up() takes 0 positional arguments but 1 was given

Error in callback <function Halo.__init__.<locals>.clean_up at 0x0000022061F64160> (for post_run_cell), with arguments args (<ExecutionResult object at 22050d0fee0, execution_count=8 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 22050d0ed10, raw_cell="def main():
    # Get user inputs
    location = i.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/mailm/Documents/Production-Ready-Multi-Agent-RAG-Project/notebooks/trip_planner_multi_ai_agent_notebook.ipynb#X15sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: Halo.__init__.<locals>.clean_up() takes 0 positional arguments but 1 was given